In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt

In [7]:
test_data = pd.read_csv('test.csv')

test_data = test_data.drop('Name', axis=1)

#Ersetze die Cabin Nummer nur mit dem zugehörigen Deck 
test_data['Cabin'] = test_data['Cabin'].str[0]

# Identify non-numeric columns
non_numeric_cols = test_data.select_dtypes(include=['object']).columns

# Apply one-hot encoding to non-numeric columns
training = pd.get_dummies(test_data, columns=non_numeric_cols, drop_first=True)

#Apply one-hot encoding to non-numeric columns
test_data = pd.get_dummies(test_data, columns=(['HomePlanet','Destination', 'Cabin' ]), drop_first=True)
test_data.info()

test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                4277 non-null   object 
 1   CryoSleep                  4184 non-null   object 
 2   Age                        4186 non-null   float64
 3   VIP                        4184 non-null   object 
 4   RoomService                4195 non-null   float64
 5   FoodCourt                  4171 non-null   float64
 6   ShoppingMall               4179 non-null   float64
 7   Spa                        4176 non-null   float64
 8   VRDeck                     4197 non-null   float64
 9   HomePlanet_Europa          4277 non-null   bool   
 10  HomePlanet_Mars            4277 non-null   bool   
 11  Destination_PSO J318.5-22  4277 non-null   bool   
 12  Destination_TRAPPIST-1e    4277 non-null   bool   
 13  Cabin_B                    4277 non-null   bool 

In [3]:
training_data = pd.read_csv('train.csv')

#Fülle NAs in den folgenden Spalten mit dem Median Wert 0.0 auf
training_data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = training_data[['RoomService', 
                                                                                              'FoodCourt', 
                                                                                              'ShoppingMall', 
                                                                                              'Spa', 
                                                                                              'VRDeck']].fillna(0.0)
#Age mit dem Durchschnitt befüllen
training_data['Age'] = training_data['Age'].fillna(training_data['Age'].mean())

#Drop Name column
training_data = training_data.drop('Name', axis=1)

#Ersetze die Cabin Nummer nur mit dem zugehörigen Deck 
training_data['Cabin'] = training_data['Cabin'].str[0]

training_data = training_data.dropna()

training_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7736 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   7736 non-null   object 
 1   HomePlanet    7736 non-null   object 
 2   CryoSleep     7736 non-null   object 
 3   Cabin         7736 non-null   object 
 4   Destination   7736 non-null   object 
 5   Age           7736 non-null   float64
 6   VIP           7736 non-null   object 
 7   RoomService   7736 non-null   float64
 8   FoodCourt     7736 non-null   float64
 9   ShoppingMall  7736 non-null   float64
 10  Spa           7736 non-null   float64
 11  VRDeck        7736 non-null   float64
 12  Transported   7736 non-null   bool   
dtypes: bool(1), float64(6), object(6)
memory usage: 793.2+ KB


In [4]:
# Turn VIP and CyroSleep into 1 and 0
training_data['VIP'] = training_data['VIP'].replace({True: 1, False: 0 })
training_data['CryoSleep'] = training_data['CryoSleep'].replace({True: 1, False: 0 })

training_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7736 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   7736 non-null   object 
 1   HomePlanet    7736 non-null   object 
 2   CryoSleep     7736 non-null   int64  
 3   Cabin         7736 non-null   object 
 4   Destination   7736 non-null   object 
 5   Age           7736 non-null   float64
 6   VIP           7736 non-null   int64  
 7   RoomService   7736 non-null   float64
 8   FoodCourt     7736 non-null   float64
 9   ShoppingMall  7736 non-null   float64
 10  Spa           7736 non-null   float64
 11  VRDeck        7736 non-null   float64
 12  Transported   7736 non-null   bool   
dtypes: bool(1), float64(6), int64(2), object(4)
memory usage: 793.2+ KB


C:\Users\tscha\AppData\Local\Temp\ipykernel_24564\2884129581.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  training_data['VIP'] = training_data['VIP'].replace({True: 1, False: 0 })
C:\Users\tscha\AppData\Local\Temp\ipykernel_24564\2884129581.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  training_data['CryoSleep'] = training_data['CryoSleep'].replace({True: 1, False: 0 })


In [5]:
#Apply one-hot encoding to non-numeric columns
training_data = pd.get_dummies(training_data, columns=(['HomePlanet','Destination', 'Cabin' ]), drop_first=True)
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7736 entries, 0 to 8692
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                7736 non-null   object 
 1   CryoSleep                  7736 non-null   int64  
 2   Age                        7736 non-null   float64
 3   VIP                        7736 non-null   int64  
 4   RoomService                7736 non-null   float64
 5   FoodCourt                  7736 non-null   float64
 6   ShoppingMall               7736 non-null   float64
 7   Spa                        7736 non-null   float64
 8   VRDeck                     7736 non-null   float64
 9   Transported                7736 non-null   bool   
 10  HomePlanet_Europa          7736 non-null   bool   
 11  HomePlanet_Mars            7736 non-null   bool   
 12  Destination_PSO J318.5-22  7736 non-null   bool   
 13  Destination_TRAPPIST-1e    7736 non-null   bool   
 1

In [6]:
#Check for imbalance
anzahl = training_data['Transported'].value_counts(normalize=False)
print(anzahl)

Transported
True     3893
False    3843
Name: count, dtype: int64


In [8]:
# Define Model
train_model = RandomForestClassifier(n_estimators=100, max_features=3, random_state=0)

# Train a random forest model
X = training_data.drop(columns=['Transported'])
y = training_data['Transported']

In [9]:
# Cross-validation
cv_fits_accuracy = cross_val_score(train_model, X, y, cv=4, scoring='accuracy')
cv_fits_precision = cross_val_score(train_model, X, y, cv=4, scoring='precision')
cv_fits_recall = cross_val_score(train_model, X, y, cv=4, scoring='recall')

print("\nCV-Accuracy:", np.mean(cv_fits_accuracy))
print("CV-Precision:", np.mean(cv_fits_precision))
print("CV-Recall:", np.mean(cv_fits_recall))


CV-Accuracy: 0.7383660806618407
CV-Precision: 0.8767731233520467
CV-Recall: 0.5882463580323772


In [10]:
# Train the final model
train_model.fit(training_data.drop(columns=['Transported']), training_data['Transported'])

RandomForestClassifier(max_features=3, random_state=0)

In [11]:
# Apply on test set
test_predictions = train_model.predict(test_data)
print(test_predictions)

[ True False  True ...  True  True False]


In [12]:
# Erstelle ein DataFrame mit den Vorhersagen und Wahrscheinlichkeiten
test_predictions_df = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],  # Identifiziere Passagiere  
    'Transported': test_predictions,  # Modellvorhersagen
})
test_predictions_df.to_csv('prediction_file.csv', index=False)